In [1]:
from keras.layers import Concatenate, Activation, Lambda, Subtract, Dot, Multiply
from keras.layers import Input, Dense, Flatten, Convolution1D, RepeatVector, Add
from keras.models import Model, Sequential
from keras.initializers import Zeros
from keras.applications.vgg19 import VGG19
from keras.activations import softmax
from keras import backend as K
import sys, glob, math, os
import cv2
import numpy as np
import re

input_size_x = [196, 512]
input_size_c = [50, 512]

Using TensorFlow backend.


In [2]:
def custom_softmax(x):
    y = softmax(x, axis=2)
    return y

In [3]:
def rep_tensor(x,i,k):
    # repeat 2d tensor x, i-th column for k times
    t = x[:,i,:]
    s = K.int_shape(t)
    z = K.repeat_elements(t, k, 1)
    z = K.reshape(z,(-1,s[1],k))
#    z = K.permute_dimensions(z,(0,1,2))
    return z

In [4]:
def make_feature_tensor(x,i,c):
    # make [xi-c(1) xi-c(2) ... xi-c(n)] tensor
    t = x[:,i,:]
    s = K.int_shape(c)
    z = K.repeat_elements(t, s[1], 1)
    z = K.reshape(z,(-1,s[1],s[2]))
    z = z - c
#    z = K.permute_dimensions(z,(0,1,2))
    return z

In [5]:
def sum_along_row(x):
    return K.sum(x,axis=1)

In [27]:
def netvlad_model(num_cluster,num_feature,num_feature_dim):
    # input feature and cluster centers
    x_input = Input(input_size_x, name='x_input')
    c_input = Input(input_size_c, name='c_input')
    
    conv_1 = []
    
    for i in range(0,num_cluster):
        x = Convolution1D(1, 1, padding='same', use_bias=True)(x_input)
        conv_1.append(x)
    
    x = Concatenate(axis=2)(conv_1)
    # now we get j(num of features) x K(num of clusters) matrix
    w = Lambda(custom_softmax)(x)
    
    yy = []
    
    for i in range(0,num_feature):
        W = Lambda(rep_tensor,arguments={'i': i, 'k': num_feature_dim})(w)
        XC = Lambda(make_feature_tensor, arguments={'i': i, 'c': c_input})(x_input)
        x = Multiply()([W,XC])
        yy.append(x)
        
    y = Add()(yy)    
        
    model = Model(inputs=[x_input,c_input], outputs=x)
    
    model.summary()
 
    return model
#END vgg19_lower_model()

In [28]:
data = np.load("data.npy")
center = np.load("centers.npy")
centers = np.zeros((data.shape[0],center.shape[0],center.shape[1]))
for i in range(0,data.shape[0]):
    centers[i,:,:]=center

me = np.load("mean.npy")
stv = np.load("stv.npy")

In [29]:
print(data.shape)
print(centers.shape)
print(me.shape)
print(stv.shape)

(184, 196, 512)
(184, 50, 512)
(512,)
(512,)


In [30]:
model = netvlad_model(centers.shape[1],data.shape[1],data.shape[2])

KeyboardInterrupt: 

In [11]:
pred = model.predict([data,centers], batch_size=32, verbose=1)

ValueError: GraphDef cannot be larger than 2GB.

In [11]:
print(pred.shape)
print(pred[0,:,:])

(184, 200, 512)
[[  3.15935457e-19   4.57456987e-19  -3.96496257e-19 ...,   2.93296044e-17
    2.90684370e-17   2.84540238e-17]
 [  2.74009026e-14   2.87115438e-14   2.87494807e-14 ...,   3.19672220e-17
   -5.52549608e-16  -5.44674531e-16]
 [ -7.58749218e-17  -3.45222148e-16  -1.62459212e-16 ...,   1.85051733e-15
    1.94547692e-15   1.80440422e-15]
 ..., 
 [ -1.21423186e-19   1.96653906e-19  -9.02374303e-20 ...,   2.70860990e-19
   -9.56628175e-19   2.30399579e-19]
 [  2.78176736e-20   3.89151514e-20   6.28470051e-20 ...,   8.25069069e-18
    8.31924080e-18   8.29974250e-18]
 [  1.93729153e-21   1.93520459e-21   1.93189648e-21 ...,  -1.95018205e-22
   -2.50086199e-23   9.58890013e-24]]


In [12]:
#from keras.utils import plot_model
#plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.